### Trabalho Final de Cocada: Aproximador de Confrontos em Smash
---  

**Aluno**:  Paulo Roberto Ferreira de Godoy Moreira  
**DRE:** 123451653  

Este jupyter foi feito como parte do trabalho final da disciplina "Computação Científica e Análise de Dados", utilizando dados coletados sobre as taxas de vitória de personagens no jogo de luta "super smash bros ultimate", e tentando aproximá-las por meio de datasets limitados. Para isso, foram usados dois conceitos vistos durante a disciplina: mínimos quadrados (para medirmos a "força" de cada personagem, e regressão logística, para medirmos a taxa de vitória de um personagem a partir de sua força). Ao final, os dados mais relevantes são salvos, e usados para alimentar um website, feito para facilitar a visualização dos resultados. 

### Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
import csv

### Dicionário de Personagens

In [2]:
# Definição dos personagens com int para string
personagens = {
    0: "mario",
    1: "donkey-kong",
    2: "link",
    3: "samus",
    4: "dark-samus",
    5: "yoshi",
    6: "kirby",
    7: "fox",
    8: "pikachu",
    9: "luigi",
    10: "ness",
    11: "captain-falcon",
    12: "jigglypuff",
    13: "peach",
    14: "daisy",
    15: "bowser",
    16: "ice-climbers",
    17: "sheik",
    18: "zelda",
    19: "dr-mario",
    20: "pichu",
    21: "falco",
    22: "marth",
    23: "lucina",
    24: "young-link",
    25: "ganondorf",
    26: "mewtwo",
    27: "roy",
    28: "chrom",
    29: "mr-game-and-watch",
    30: "meta-knight",
    31: "pit",
    32: "dark-pit",
    33: "zero-suit-samus",
    34: "wario",
    35: "snake",
    36: "ike",
    37: "pokemon-trainer",
    38: "diddy-kong",
    39: "lucas",
    40: "sonic",
    41: "king-dedede",
    42: "olimar",
    43: "lucario",
    44: "rob",
    45: "toon-link",
    46: "wolf",
    47: "villager",
    48: "mega-man",
    49: "wii-fit-trainer",
    50: "rosalina",
    51: "little-mac",
    52: "greninja",
    53: "palutena",
    54: "pac-man",
    55: "robin",
    56: "shulk",
    57: "bowser-jr",
    58: "duck-hunt",
    59: "ryu",
    60: "ken",
    61: "cloud",
    62: "corrin",
    63: "bayonetta",
    64: "inkling",
    65: "ridley",
    66: "simon",
    67: "richter",
    68: "king-k-rool",
    69: "isabelle",
    70: "incineroar",
    71: "piranha-plant",
    72: "joker",
    73: "hero",
    74: "banjo-and-kazooie",
    75: "terry",
    76: "byleth",
    77: "min-min",
    78: "steve",
    79: "sephiroth",
    80: "pyra-and-mythra",
    81: "kazuya",
    82: "sora"
}

# Definição dos personagens com string para int ("desfaz" o anterior)
personagens_inv = {value: key for key, value in personagens.items()}

### Funções Auxiliares

In [3]:
# Função para montar o sistema com base em um dataframe de entrada
def montaSistema(df):
    # Pegando quantidade de confrontos e de personagengs
    qtd_confrontos = len(df['Personagem1'])
    qtd_personagens = len(personagens)

    # Definindo a matriz e o vetor que definem o sistema
    A = np.zeros((qtd_confrontos, qtd_personagens))
    b = np.zeros((qtd_confrontos))

    for i in range(qtd_confrontos):
        # Pegando dados a partir do dataframe
        personagem1 = personagens_inv[df['Personagem1'].iloc[i]]
        personagem2 = personagens_inv[df['Personagem2'].iloc[i]]
        taxa1 = (df['taxaVitoria1'].iloc[i])/100
        taxa2 = (df['taxaVitoria2'].iloc[i])/100

        # Montando a matriz com -1 e +1 nos personagens envolvidos
        A[i, personagem1] = 1
        A[i, personagem2] = -1
        
        # Montando vetor de "resposta" com a diferença das taxas de vitória
        b[i] = taxa1 - taxa2
    
    return A, b

In [4]:
# Função que calcula a taxa de vitória geral de um personagem, 
# A partir de parâmetros x, alpha e beta. 
def regressaoLogistica(x, alpha, beta):
    return 100 * (np.exp(x*alpha + beta)/(1 + np.exp(x*alpha + beta)))


In [5]:
# Função que calcula a taxa de vitória do personagem que vence o confronto
# Ela não mostrará nenhuma mensagem, e é usada
# Para montar os dados que irão para o site
def taxaVitoriaSemMensagem(approx, a, b):

    # "Delta" é a distância entre as forças de cada personagem
    # Esse será o parâmetro utilizado para calcular a taxa de vitória
    # Desse confronto, por meio da regressão logística. 
    delta = abs(approx[personagens_inv[a]] - approx[personagens_inv[b]])
    taxa = regressaoLogistica(delta, 2.0001, 0)

    # Checamos se o personagem vitorioso foi o primeiro ou o segundo, a 
    # Partir da força. Se for o segundo, então a taxa de vitória 
    # Será 100 - taxa.
    if(approx[personagens_inv[a]] < approx[personagens_inv[b]]):
        taxa = (100 - taxa)

    return taxa

# Função que calcula a taxa de vitória em um confronto direto 
# Entre dois personagens, mostrando quem vence quem, e 
# As respectivas chances de vitória. Para facilitar, a função 
# taxaVitoriaSemMensagem é chamada, e então apenas colocamos
# A mensagem correta.  
def taxaVitoriaAproximada(approx, a, b):
    taxa = taxaVitoriaSemMensagem(approx, a, b)
    print("Taxa de Vitória Aproximada:")
    if (taxa == 50.00):
        print("\tEmpate!")
    elif (taxa > 50.00):
        print("\t", a, "vence", b, "({:.2f}%".format(taxa), " -- ", "{:.2f}%)".format(100-taxa))
    else:
        print("\t", b, "vence", a, "({:.2f}%".format(100-taxa), " -- ", "{:.2f}%)".format(taxa))
    print("\n")

# Função que recupera, em um dataframe de confrontos, as taxas
# De confronto entre dois personagens. É utilizada apenas 
# Para comparar resultados com a função
def taxaVitoriaReal(df, a, b):
    resultado = tuple(df[((df["Personagem1"] == a) & (df["Personagem2"] == b)) | ((df["Personagem1"] == b) & (df["Personagem2"] == a))].iloc[0])

    print("Taxa de Vitória Real:")
    if(resultado[2] == resultado[3]):
        print("\tEmpate!")
    elif(resultado[2] > resultado[3]):
        print("\t", resultado[0], "vence", resultado[1], "({:.2f}%".format(resultado[2]), " -- ", "{:.2f}%)".format(resultado[3]))
    else:
        print("\t", resultado[1], "vence", resultado[0], "({:.2f}%".format(resultado[3]), " -- ", "{:.2f}%)".format(resultado[2]))

In [6]:
# Função que calcula o erro absoluto entre dois vetores x e y. 
# Para isso, ele simplesmente calcula a soma das distância item-a-item
def erroAbsoluto(x, y):
    sum = 0
    for i in range(len(x)):
        sum += abs(x[i] - y[i])
    return sum

# Função que calcula o erro absoluto médio entre dois vetores x e y. 
# Para isso, ele simplesmente calcula a soma das distância item-a-item 
# Então, a divide pelo tamanho dos vetores. 
def erroAbsolutoMedio(x, y):
    sum = 0
    for i in range(len(x)):
        sum += abs(x[i] - y[i])
    sum = sum/len(x)
    return sum

# Função que calcula o erro relativo entre dois vetores x e y. 
# Isto é, o cosseno entre os dois vetores (produto interno/produto das normas).
def erroRelativo(x, y):
    return (np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))

### Importando Dados

In [7]:
# Nome dos arquivos de entrada, e o caminho em que eles estão:
arquivoConfrontos = "matchups1.csv"
arquivoPersonagens = "charactersTodos.csv"

diretorioConfrontos = "../data/entradas_matchups"
diretorioPersonagens = "../data/entradas_personagens"

arquivoConfrontos = os.path.join(diretorioConfrontos, arquivoConfrontos)
arquivoPersonagens = os.path.join(diretorioPersonagens, arquivoPersonagens)

In [8]:
# Lendo os dados de entrada de confrontos
confrontos_dataframe = pd.read_csv(arquivoConfrontos, decimal = '.')
confrontos_dataframe

,Personagem1,Personagem2,taxaVitoria1,taxaVitoria2
0,mario,donkey-kong,45.51,54.49
1,mario,link,48.53,51.47
2,mario,samus,45.39,54.61
3,mario,dark-samus,43.72,56.28
4,mario,yoshi,46.22,53.78
...,...,...,...,...
2522,simon,richter,49.27,50.73
2523,simon,king-k-rool,59.18,40.82
2524,simon,kazuya,50.42,49.58
2525,simon,sora,59.71,40.29


In [9]:
personagens_dataframe = pd.read_csv(arquivoPersonagens)
personagens_dataframe

,Personagem,taxaVitoria,taxaDerrota
0,mario,47.75,52.25
1,donkey-kong,52.02,47.98
2,link,49.06,50.94
3,samus,52.69,47.31
4,dark-samus,52.86,47.14
...,...,...,...
78,steve,58.15,41.85
79,sephiroth,44.48,55.52
80,pyra-and-mythra,46.16,53.84
81,kazuya,54.12,45.88


Também montamos um vetor com as taxas de vitória reais para cada personagem, que será utilizado depois para o cálculo do erro:

In [10]:
taxas_vitorias_reais = np.zeros(len(personagens_dataframe["taxaVitoria"]))
for i in range(len(personagens_dataframe["taxaVitoria"])):
    taxas_vitorias_reais[i] = personagens_dataframe["taxaVitoria"].iloc[i]

### Aproximando Forças

Para aproximarmos as forças, usamos mínimos quadrados com os dados dos confrontos; Para isso, usamos a função pré-estabelecida do python, "np.linalg.lstsq".

O sistema linear sendo resolvido é A * f = b, onde A é a matriz dos confrontos (possui valores -1 e +1 nos dois personagens envolvidos) e b é o vetor das diferenças das porcentagens de vitória entre os dois personagens. 

In [11]:
A, b = montaSistema(confrontos_dataframe)
forca, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

for f in forca:
    print(f)

-0.051100997246757755
0.04172198739278847
-0.010975782650965493
0.05310495137719673
0.06744612771774616
0.009640103228997316
-0.09036589666187729
-0.08274667833567698
-0.03927678544807722
0.0945244347116417
0.04282468661691105
-0.065161402670041
-0.10562775760734759
0.01674403128906195
-0.032948948136053355
-0.012583614802720099
0.036003519567446775
-0.17361236407389075
0.04698961561424764
0.0750427801712093
-0.036035700841448526
-0.0655470932465121
-0.198747093246512
-0.21009646033511928
-0.04395721982879071
-0.0466499459982191
-0.09347733757483043
-0.12331350897052099
-0.07545709871411174
0.017687331561936612
0.01352633310154222
-0.10476840374056316
-0.10790276176050043
-0.033203047633114366
-0.037039388632238976
0.14732610545670638
0.07524604510012547
-0.02975755739385226
-0.0014258818566117368
-0.023059641845469075
0.0751266595243944
0.08802170598172969
0.008873760776249781
-0.06346288649562262
0.11593437377834974
0.014798671311699263
-0.05772610805107894
-0.016932783597672064
0.05

### Aproximando Vitórias

Para aproximarmos a taxa de vitória geral, usamos regressão logística com as forças calculadas.

In [12]:
quantidade_personagens = len(forca)
taxas_vitorias = np.zeros(quantidade_personagens)

for i in range(quantidade_personagens):
    
    # Valores 2.0001 e 0 para alpha e beta foram encontrados simplesmente aplicando a função algumas vezes, e vendo 
    # Parâmetros que aproximariam bem a função. 
    taxas_vitorias[i] = regressaoLogistica(forca[i], 2.0001, 0) 

In [13]:
print("Personagem -- Taxa de Vitória Geral\n")
for i in range(quantidade_personagens):
    print(personagens[i], " -- ", "{:.2f}%".format(taxas_vitorias[i]))

Personagem -- Taxa de Vitória Geral

mario  --  47.45%
donkey-kong  --  52.08%
link  --  49.45%
samus  --  52.65%
dark-samus  --  53.37%
yoshi  --  50.48%
kirby  --  45.49%
fox  --  45.87%
pikachu  --  48.04%
luigi  --  54.71%
ness  --  52.14%
captain-falcon  --  46.75%
jigglypuff  --  44.74%
peach  --  50.84%
daisy  --  48.35%
bowser  --  49.37%
ice-climbers  --  51.80%
sheik  --  41.41%
zelda  --  52.35%
dr-mario  --  53.75%
pichu  --  48.20%
falco  --  46.73%
marth  --  40.19%
lucina  --  39.65%
young-link  --  47.80%
ganondorf  --  47.67%
mewtwo  --  45.34%
roy  --  43.87%
chrom  --  46.23%
mr-game-and-watch  --  50.88%
meta-knight  --  50.68%
pit  --  44.78%
dark-pit  --  44.63%
zero-suit-samus  --  48.34%
wario  --  48.15%
snake  --  57.31%
ike  --  53.76%
pokemon-trainer  --  48.51%
diddy-kong  --  49.93%
lucas  --  48.85%
sonic  --  53.75%
king-dedede  --  54.39%
olimar  --  50.44%
lucario  --  46.83%
rob  --  55.77%
toon-link  --  50.74%
wolf  --  47.12%
villager  --  49.15%
m

### Erro da Aproximação

Aqui, checamos o erro da aproximação de duas formas diferentes: vendo o cosseno entre os vetores, e também vendo o erro absoluto médio (o quanto cada item está distante, em média, de seu valor real):

In [14]:
print("Erro relativo: cos(theta) = ", erroRelativo(taxas_vitorias, taxas_vitorias_reais))
print("Erro absoluto: +-", erroAbsoluto(taxas_vitorias, taxas_vitorias_reais))
print("Erro absoluto médio: +-", erroAbsolutoMedio(taxas_vitorias, taxas_vitorias_reais))

Erro relativo: cos(theta) =  0.9999746414728219
Erro absoluto: +- 26.955289559552433
Erro absoluto médio: +- 0.32476252481388473


### Confrontos

Agora, vamos tentar descobrir as chances de vitória de cada personagem em um confronto direto, considerando as aproximações de força deles!

In [15]:
personagem1 = "mario"
personagem2 = "donkey-kong"
taxaVitoriaAproximada(forca, personagem1, personagem2)
taxaVitoriaReal(confrontos_dataframe, personagem1, personagem2)

Taxa de Vitória Aproximada:
	 donkey-kong vence mario (54.63%  --  45.37%)


Taxa de Vitória Real:
	 donkey-kong vence mario (54.49%  --  45.51%)


### Salvando dados

Para montar o site que mostrará os resultados dos confrontos e o ranking geral dos personagens, é necessário guardar um arquivo com os nomes dos personagens, suas forças (resultado de mínimos quadrados), e suas taxas gerais de vitória (resultado da regressão logística). Também é adicionado um vetor com códigos hex para uma cor específica para cada personagem, que será utilizada no ranking do site. 

In [16]:
# Vetor com as cores
colors = [
    "#C00000",
    "#814E1D",
    "#0A7C46",
    "#e89410",
    "#573d71",
    "#6ba57a",
    "#c27ba0",
    "#cf9f55",
    "#7f6000",
    "#7ee051",
    "#e75050",
    "#d0be22",
    "#e521e5",
    "#e89ce8",
    "#e8d39c",
    "#8a6658",
    "#7695b2",
    "#8948d2",
    "#522f79",
    "#d2291e",
    "#ffd966",
    "#a29f9f",
    "#68b3ca",
    "#a9c3cc",
    "#82e555",
    "#3e1a52",
    "#aa75c7",
    "#c85816",
    "#2b16c8",
    "#4c4444",
    "#9f7b9d",
    "#dbd825",
    "#43470c",
    "#5464f0",
    "#e5dc4a",
    "#57724a",
    "#729abf",
    "#e82727",
    "#a46418",
    "#b0ac62",
    "#233f85",
    "#d7a5a5",
    "#baafaf",
    "#5b8dba",
    "#bcbcbc",
    "#65ff6b",
    "#66588a",
    "#ea9999",
    "#2986cc",
    "#d1d8df",
    "#9fc5e8", 
    "#9fe8c2",
    "#004bcc",
    "#037412",
    "#d6e700",
    "#8e7cc3",
    "#c52626", 
    "#8fce00",
    "#ffd966",
    "#894141",
    "#ca5d5d",
    "#676767", 
    "#cacaca",
    "#20124d",
    "#eab11f", 
    "#720564",
    "#714b0f",
    "#134f5c",
    "#057218", 
    "#f2ff4f",
    "#d86448",
    "#5caa46",
    "#c70000",
    "#c425cc",
    "#a2cc25",
    "#e5b1b1",
    "#33dbb9",
    "#bf9000",
    "#85afb7",
    "#999999",
    "#c069a5",
    "#5e5e5e",
    "#cc6756",
]

In [17]:
# Vetor que guarda cada uma das informações necessárias, para cada personagem
guardar_personagens = []
for i in range(quantidade_personagens):
    guardar_personagens.append([personagens[i], forca[i], taxas_vitorias[i], colors[i]])

In [18]:
# Então, o arquivo com o nome especificado é criado na pasta saída, e 
# Cada um dos itens do vetor anterior é escrito nele, seguindo a 
# Formatação "name", "forca", "percentage", "color". 

arquivoSaida = "saida.csv"
diretorioSaida = "../data/saidas"
arquivoSaida = os.path.join(diretorioSaida, arquivoSaida)

with open(arquivoSaida, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['name', 'forca', 'percentage', 'color'])
    writer.writerows(guardar_personagens)

Abaixo, temos dois códigos comentados, ambos usados para montar o csv do erro. O primeiro é utilizado, apenas, para escrever os nomes das colunas (em modo de sobrescrita). Já o segundo é utilizado no modo append, servindo para escrita dos erros obtidos durante o código atual. A montagem do csv em si foi feita rodando a primeira parte uma vez, e a segunda diversas vezes, uma para cada uma das entradas. 

In [19]:
# arquivoErros = "erros.csv"
# diretorioErros = "../data/erros"
# arquivoErros = os.path.join(diretorioErros, arquivoErros)

# with open(arquivoErros, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['subset', 'erroRelativo', 'erroAbsoluto', 'erroAbsolutoMedio'])

In [20]:
# arquivoErros = "erros.csv"
# diretorioErros = "../data/erros"
# arquivoErros = os.path.join(diretorioErros, arquivoErros)


# guardar_erros = []
# guardar_erros.append([subset, erroRelativo(taxas_vitorias, taxas_vitorias_reais), erroAbsoluto(taxas_vitorias, taxas_vitorias_reais), erroAbsolutoMedio(taxas_vitorias, taxas_vitorias_reais)])

# with open(arquivoErros, mode='a', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(guardar_erros)